In [1]:
from google.colab import files
import pickle

uploaded = files.upload()


Saving tweets-data.csv to tweets-data.csv


In [2]:
# Install Vader
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.7 MB/s eta 0:00:00


In [5]:
import pandas as pd

df = pd.read_csv('tweets-data.csv')
df.head()  # Pour vérifier les colonnes

,Unnamed: 0,Date Created,Number of Likes,Source of Tweet,Tweets,hashtag
0,0,2023-06-25 19:16:20+00:00,0,NaN,@jacksonhinklle #wagner with 6.2 billion dolla...,wagner
1,1,2023-06-25 19:16:18+00:00,0,NaN,Pobrecito es discapacitado\n#Reddetuiterosdemo...,wagner
2,2,2023-06-25 19:16:07+00:00,0,NaN,News from the EIR Daily Alert\n\n“#Putin Addre...,wagner
3,3,2023-06-25 19:15:56+00:00,0,NaN,It's Messi day #Messi𓃵 #Messi36 #Russia #bigst...,wagner
4,4,2023-06-25 19:15:54+00:00,0,NaN,Il passaggio chiave di Machiavelli era questo ...,wagner


In [7]:
print(df.columns)


Index(['Unnamed: 0', 'Date Created', 'Number of Likes', 'Source of Tweet',
       'Tweets', 'hashtag'],
      dtype='object')


In [8]:
def get_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(str(text))
    compound = scores['compound']

    if compound >= 0.05:
        sentiment = 'positive'
    elif compound <= -0.05:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'

    return sentiment, compound

df[['sentiment', 'score']] = df['Tweets'].apply(lambda x: pd.Series(get_sentiment(x)))
df[['Tweets', 'sentiment', 'score']].head()


,Tweets,sentiment,score
0,@jacksonhinklle #wagner with 6.2 billion dolla...,positive,0.4939
1,Pobrecito es discapacitado\n#Reddetuiterosdemo...,neutral,0.0000
2,News from the EIR Daily Alert\n\n“#Putin Addre...,positive,0.2960
3,It's Messi day #Messi𓃵 #Messi36 #Russia #bigst...,neutral,0.0000
4,Il passaggio chiave di Machiavelli era questo ...,neutral,0.0000


In [9]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_tweet(text):
    # 1. Convertir en string
    text = str(text)

    # 2. Enlever les mentions, hashtags, URLs
    text = re.sub(r'@\w+', '', text)          # mentions @user
    text = re.sub(r'#\w+', '', text)          # hashtags
    text = re.sub(r'http\S+|www\S+', '', text)  # URLs

    # 3. Enlever tout sauf lettres (et espaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 4. Mettre en minuscule
    text = text.lower()

    # 5. Tokeniser (séparer en mots)
    tokens = text.split()

    # 6. Supprimer les stop words
    tokens = [word for word in tokens if word not in stop_words]

    # 7. Recomposer une phrase avec les tokens restants
    cleaned_text = ' '.join(tokens)

    return cleaned_text
df['cleaned_text'] = df['Tweets'].apply(clean_tweet)
df[['Tweets', 'cleaned_text']].head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Tweets,cleaned_text
0,@jacksonhinklle #wagner with 6.2 billion dolla...,billion dollar
1,Pobrecito es discapacitado\n#Reddetuiterosdemo...,pobrecito es discapacitado
2,News from the EIR Daily Alert\n\n“#Putin Addre...,news eir daily alert addressed people armed ju...
3,It's Messi day #Messi𓃵 #Messi36 #Russia #bigst...,messi day
4,Il passaggio chiave di Machiavelli era questo ...,il passaggio chiave di machiavelli era questo ...


In [10]:
df[['cleaned_sentiment', 'cleaned_score']] = df['cleaned_text'].apply(
    lambda x: pd.Series(get_sentiment(x))
)
df[['cleaned_text', 'cleaned_sentiment', 'cleaned_score']].head()


,cleaned_text,cleaned_sentiment,cleaned_score
0,billion dollar,neutral,0.000
1,pobrecito es discapacitado,neutral,0.000
2,news eir daily alert addressed people armed ju...,positive,0.296
3,messi day,neutral,0.000
4,il passaggio chiave di machiavelli era questo ...,neutral,0.000
